In [2]:
import numpy as np
import cv2
from subprocess import check_output
from sklearn.model_selection import KFold
from sklearn import metrics
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Input, Flatten, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
def get_test_set_files_paths_arr_classes():
    fpa = []
    fpa.append(check_output(["ls", "seedlings_data/test/"]).decode("utf8").strip().split("\n"))
    filez = np.array(fpa)
    classes = np.array(check_output(["ls", "seedlings_data/train"]).decode("utf8").strip().split("\n"))
    return filez, classes

In [4]:
def get_predicted_classes(classes, predictions_vals): 
    max_indexes = np.argmax(predictions_vals, axis=1)
    pred_class = classes[max_indexes]
    pred_class = np.expand_dims(pred_class, axis=0)
    return pred_class


In [5]:
def get_subm_result(files_names_arr, classes_pred):
    conc = np.concatenate((files_names_arr, classes_pred), axis=0)
    conc = conc.astype('str')
    return np.apply_along_axis(lambda d: d[0] + ',' + d[1], 0, conc)

In [6]:
preds = np.load('seedl_subms/InceptionV3_preds_kfold10ep25_cat_cross_ent_0.98236.npy')
preds2 = np.load('seedl_subms/ResNet50_preds_kfold10ep25_cat_cross_ent_0.97481.npy')

predsx = (preds + preds2) / 2

filez, classes = get_test_set_files_paths_arr_classes()

predicted_classes = get_predicted_classes(classes, predsx)

res = get_subm_result(filez, predicted_classes)


res.tofile("seedl_subms/preds_avg.csv", sep='\n')


